In [1]:
# Import required libraries
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
#plt.rcParams['pdf.fonttype'] = 42
import pandas as pd
from IPython.core.display import HTML
import numpy as np
%matplotlib qt
import glob
import readData
reload(readData) #avoid kernel restart if changed
import re
import databaseQuery

In [1]:
## Read in the data as a pandas dataframe
# Files should be .csv format in \Data\ subdirectory
# Files should be simply named SAGE_SOUTH or RIV_NORTH, without numerals
allFiles = glob.glob("Data\*.csv") #get all files with csv extension in this subfolder
print allFiles


#Return pandas dataframe with all the data, datetime index
#toggle saveCSV True/False to save a csv of the compiled data

voltage, radius = readData.readCSV(allFiles, saveCSV = False) 


NameError: name 'glob' is not defined

In [ ]:
radius.head()

In [ ]:
## Read in the metadata as a pandas dataframe
metadata = pd.read_csv('PISTON_DENDROMETER_METADATA.csv',sep=',', dtype=str)

#Create dictionaries between sensor ID and metadata info
Site = dict(zip(metadata['Micron Displacement DSID'], metadata['Site'] )) 
Slope = dict(zip(metadata['Micron Displacement DSID'], metadata['Slope'] )) 
Species = dict(zip(metadata['Micron Displacement DSID'], metadata['Species'] )) 
Tree = dict(zip(metadata['Micron Displacement DSID'], metadata['Tree #'] )) 


In [ ]:
metadata.head()

In [ ]:
#%% SPECIFY TIME RANGE OF SERIES
startDateTime = pd.to_datetime('2015-08-01 05:00:00')
stopDateTime = pd.to_datetime('2016-03-18 05:00:00')
radiusTR = radius[(pd.to_datetime(radius.index) > startDateTime) & 
                (pd.to_datetime(radius.index) < stopDateTime)]


In [ ]:
## Import from mySQL database 
#Convert datetime stamps to strings for the sql query
dateStart = str(startDateTime.year) + '-' + str(startDateTime.month) + '-' + str(startDateTime.day)
dateStop =  str(stopDateTime.year) + '-' + str(stopDateTime.month) + '-' + str(stopDateTime.day)

#here we make use of external .py file with odmquery function (thanks Collin!) to get data from the database
#1672 = cumulative WY precip, in mm
#1785 = Rainfall mm TB4 WSAM RWS_Rain_TB4_Tot2015-03-26 (ongoing)Rainfall mmAngelo Meadow WSwsam0 to 700TB4mm-L Rain Gage
#2958 = Barometric Pressure, WSSR, mBar
#2959 = Rainfall, WSSR, mm
#3048 = Rainfall Cumulative mm Sagehorn
# atmP = databaseQuery.odmquery(dateStart, dateStop, '2958', True)
# atmP.plot()
#rain = databaseQuery.odmquery(dateStart, dateStop, '2957', True)



In [ ]:
## Normalize values
radiusMin = radiusTR.apply(np.min)

#center to zero
radiusZeroed =  radiusTR-radiusMin



In [ ]:
# fig, ax = plt.subplots(2, sharex=True, figsize=(17, 22))

fig = plt.figure(figsize=(20,10))
# Get unique names of species
uniq = list(set(metadata['Species']))

# Set the color map to match the number of species
# z = range(1,len(uniq))
hot = plt.get_cmap('hot')
cNorm  = colors.Normalize(vmin=0, vmax=len(uniq))
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=plt.get_cmap('hsv') )


for column in radiusZeroed:
#         plt.figure()
    ax = plt.plot(radiusZeroed.index, radiusZeroed[column], 
             color=scalarMap.to_rgba(uniq.index(Species[column])), 
             label = (Tree[column] + ' ' + Species[column] + ' ' + Site[column]))
    plt.legend(bbox_to_anchor=(1, 1))
plt.ylabel('Radial displacement of outer live bark away from tree [microns]')
plt.ylim([0,3000])
#plt.savefig('pistonDendro.pdf', format='pdf')

$$a + b / 3$$